In [1]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import sys
sys.path.append('../..')
from functions.db_inserts import *
from functions.aws_functions import *
from functions.select_cols_functions import *

In [3]:
env_mode = 'prd' # 'dev' ou 'prd'

### Lendo arquivo fonte

In [4]:
bucket_name = "adoptai-trusted-dev"

In [5]:
if env_mode == 'dev':
    df_trusted = pd.read_csv('../../../data/train_2_classes.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = ['train_2_classes.csv', df_trusted]
elif env_mode == 'prd':
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Parâmetros de gravação

In [6]:
nome_fonte = s3[0]
nome_modelo = 'SVC Linear - gamma Scale'
parameters = {
        'C': [
            0.05, 
            0.1 
        #     0.5
        ]
}

### Preparativos para executar o modelo

In [7]:
# Separar as features (X) e os rótulos (y)
x = df_trusted.iloc[:, :-1]
y = df_trusted.iloc[:, -1]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

###  Rodando modelo KNN

In [9]:
accuracy = []
precision = []
recall = []
f1 = []

start_times = []
end_times = []

In [10]:
for c in parameters['C']:
    print(f'{time.time()} Iniciando execução: c = {c}')

    # Treinar o modelo KNN
    start_time = start_times.append(time.time())
    # svc = LinearSVC(C=c)
    svc = SVC(kernel='linear', C=c, gamma='scale')
    svc.fit(x_train, y_train)

    y_pred = svc.predict(x_test)

    # Avaliar o modelo
    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average=None))
    recall.append(recall_score(y_test, y_pred, average=None))
    f1.append(f1_score(y_test, y_pred, average=None))
    end_times.append(time.time())

    print(f'{time.time()} Finalizando execução: c = {c}')

1723407369.3354542 Iniciando execução: c = 0.05
1723407414.1792324 Finalizando execução: c = 0.05
1723407414.1792324 Iniciando execução: c = 0.1
1723407449.866659 Finalizando execução: c = 0.1


### Obtendo métricas de desempenho

In [11]:
metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

### Gravando dados no banco

In [12]:
model = {'nome_modelo': nome_modelo, 'nome_fonte': nome_fonte}
execution = {'accuracy': accuracy, 'start_time': start_times, 'end_time': end_times}
hyperparams = parameters
desempenho = {'precision_values': metrics['precision'], 'recall_values': metrics['recall'], 'f1_values': metrics['f1_score']}

In [13]:
insert_values(model, execution, hyperparams, desempenho, env=env_mode)

Executando em dev
Registros inseridos
